# The role of algorithms in Computing

## Algorithms as a technology
### Efficiency
>For a concrete example, let us pit a faster computer (computer A) running insertion sort against a slower computer (computer B) running merge sort. They each must sort an array of 10 million numbers. (Although 10 million numbers might seem like a lot, if the numbers are eight-byte integers, then the input occupies about 80 megabytes, which fits in the memory of even an inexpensive laptop com-puter many times over.) Suppose that computer A executes 10 billion instructions per second (faster than any single sequential computer at the time of this writing)
and computer B executes only 10 million instructions per second, so that computer A is 1000 times faster than computer B in raw computing power. To make the difference even more dramatic, suppose that the world’s craftiest programmer codes insertion sort in machine language for computer A, and the resulting code requires $2n^2$ instructions to sort n numbers. Suppose further that just an average programmer implements merge sort, using a high-level language with an inefficient compiler, with the resulting code taking $50n\log_2(n)$ instructions.

In [ ]:
n = 10^7  # The size of our input
speed_A = 10^10 * 3600  # in hours
speed_B = 10^7 * 60  # in minutes

performance_A(n) = 2n^2  # significantly worse!
performance_B(n) = 50n*log2(n)  # efficient!

time_a = round(performance_A(n) / speed_A)
time_b = round(performance_B(n) / speed_B)

println("
    Algorithm A uses $time_a hours to run, whereas
    algorithm B needs $time_b minutes!")

**Exercises**
* Suppose we are comparing implementations of insertion sort and merge sort on the same machine. For inputs of size n, insertion sort runs in $8n^2$ steps, while merge sort runs in $64n\log_2(n)$ steps. For which values of n does insertion sort beat merge sort?

In [ ]:
insertion(n) = 8n^2
merge_sort(n) = 64n*log2(n)
n = 2
while insertion(n) < merge_sort(n)
    n += 1
end
println("while n < $n, insertion is faster")

* What is the smallest value of n such that an algorithm whose running time is $100n^2$ runs faster than an algorithm whose running time is $2^n$ on the same machine?

In [ ]:
alg_a(n) = 100n^2
alg_b(n) = 2^n
n = 2
while alg_a(n) > alg_b(n)
    n += 1
end
println("From n > $n, algorithm a is faster")

### Comparison of running times
For each function $f(n)$ and time $t$ in the following table, determine the largest size $n$ of a problem that can be solved in time $t$, assuming that the algorithm to solve the problem takes $f(n)$ microseconds.

### Computing $\log_2(n)$
Basically $log_2(n)$ hits the float overflow ($10^{308}$) rigth for the shortest running time. The max inputs one can process in a given time are:  
In one second ($10^6\mu\text{s}$): $2^{10^6}$. Already a quite big number  
In one minute ($6\times 10^7\mu\text{s}$):  $2^{6\times 10^7}$  
In one  hour ($3.6\times 10^9\mu\text{s}$):  $2^{10^9}\times 2^{3.6}$  
In one day ($8.64\times 10^{10}\mu\text{s}$): $2^{10^{10}}\times 2^{8.64}$  
In one month ($2.59\times 10^{12}\mu\text{s}$): $2^{10^{12}}\times 2^{2.59}$  
In one year ($3.11\times 10^{13}\mu\text{s}$): $2^{10^{13}}\times 2^{3.11}$  
In one century ($3.11\times 10^{15}\mu\text{s}$):-> $2^{10^{15}}\times 2^{3.11}$

### Computing sqrt(n), n, n^2, n^3 & 2^n times

In [ ]:
periods = ["second", "minute", "hour", "day", "month", "year", "century", ]
# Create the factors by which the times' array will grow.
factors = [60, 60, 24, 30, 12, 100, ]
# Define the starting point of the times array
times = [1e6, ]
for factor in factors
    push!(times, times[end]*factor)
end

# Sizes for sqrt(n), n, n^2, n^3 & 2^n
[times .^ 2, times, sqrt.(times), times .^ (1/3), log2.(times)]

### Computing $\text{n} \log_2(n)$  
**Newton's method**  
Computing $n\log_{2}n$ is tricky. Someone else went through it at [math.stackexchange](https://math.stackexchange.com/questions/1301343/how-to-find-the-inverse-of-n-log-n). Basically we'll use Newton method that is used to find roots, but in this case, we'll want to find the point $(x_n, 10^6)$, ie, the size of our input when it hits the $10^6\mu\text{s}$ value. It was confusing the $-10^6$ until relized that in this case we are not looking for a root but instead for the $10^6$ intercept.

In [ ]:
function newton(n::Float64, y_0::Float64)
    """
    Compute one iteration of the Newton approximation method.
    
    args: 
    n: the approximation of the size of the input in the current loop
    y_0: the target time where n should intercept
    """
    return n - ((n*log2(n)- y_0)/(log2(n)+(1/log(2))))
end

function start(y_0::Float64)
    """
    Approximate the starting point for first iteration.
    
    Args:
    y_0: the target time where n should intercept
    """
    return y_0 / log2(y_0)
end

function iterNewton(y_0::Float64)
    """
    Iterate the newton method until finding a suitable solution. 
    
    Args:
    y_0: the target time where n should intercept
    """
    prior = y_0
    approximation = start(y_0)
    loops = 0
    while abs(prior - approximation) > 1
        prior = approximation
        approximation = newton(approximation, y_0)
        loops += 1
    end
    
    # Ensure our approximations are good enough
    @assert abs(approximation * log2(approximation) - y_0) < 1
    
    return trunc(Int, approximation), loops
end

for (p, t) in zip(periods, times)
    n, loops = iterNewton(t)
    println("1 $p ($t seconds): $n ($loops loops)")
end

**Binary search**  
One can use a binary search to *efficiently* brute force estimate the size of the input

In [ ]:
function binarySearch(target::Float64)
    """
    Search the size of the input to reach the target time.
    
    Args:
    target: the time for which we are searching the input size.
    """
    lower = loops = 0
    upper = target
    while abs(lower - upper) > 1
        mid_point = (upper + lower) / 2
        p = mid_point * log2(mid_point)
        loops += 1
        if p > target
            upper = mid_point
        else
            lower = mid_point
        end
    end
    # Ensure our approximations are good enough
    @assert abs(upper * log2(upper) - target) < 20
    return trunc(Int, lower), loops
end

for (p, t) in zip(periods, times)
    n, loops = binarySearch(t)
    println("1 $p ($t seconds): $n ($loops loops)")
end

**Efficiency of above two approaches**  


In [ ]:
using BenchmarkTools
@benchmark iterNewton(times[end])

In [ ]:
@benchmark binarySearch(times[end])

As expected Newton is faster (about 3.5 times more) because of the less iterations to reach a nice approximation [52 vs 4].

### Computing $n!$

In [ ]:
for (p, t) in zip(periods, times)
    n = 1
    while factorial(n) < t
       n += 1 
    end
    println("1 $p ($t seconds): $n")
end

# Getting started
A short introduction over the main topics

**Outline**  
* Insertion sort
* Analyzing algorithms: introducing running time notation
* Design of algorithms: divide-and-conquer and merge sort

## Insertion sort
Iterate all the elements.  
Current element marks the switch point between the sorted array and the random one. So pick it.  
Starting at the last of the sorted elements `i = j - 1` shift one position to the right until reaching the first place `i=0` or (`i=-1` for 0-indexed) or a lower value `A[i] > key`  
Finally insert at the selected position (be aware of 1-indexed or 0-indexed)

In [ ]:
function insertionSort(A::Array)
    for j in 2:size(A)[1]  # Iterate all the elements
        key = A[j]  # current processed element
        # Starting at the last of all the previous elements (the already sorted
        # subarray), shift all the elements one position to the right until 
        # reaching the first place (i = 0) or a lower element (A[i] > key).
        i = j - 1  # Start at the last of the previous elements
        while i > 0 && A[i] > key
            A[i + 1] = A[i]  # shift one pos to the right
            i -= 1
        end
        # finally insert current element at the right position. Notice that i
        # can be at 0 (on exiting the while loop when i > 0) so as Julia is 
        # 1-indexed i should be increased
        A[i+1] = key
    end
    return A
end
insertionSort([5, 2, 4, 6, 1, 3, ])

**Loop invariant**
The sub-array that is already sorted (and therefore does not vary). We use this property to prove that the algorithm works (even without running it), let's see how:
- **Initialization:** in the first iteration (`j = 2`) the subarray has one element and obviously is sorted.
- **Maintenance:** over loops --`for`-- the elements are moved sequentially so order --whatever would be-- is preserved and current element is inserted in the right position.
- **Termination:** Since we iterate all the elements in the array, input and ouput sizes are the same, we must conclude that all the elements are sorted.

**Short circuiting operators**  
In `firstExpr && SecondExpr` & `firstExpr || SecondExpr`, `&&` & `||` are short circuiting operators meaning that in order to evaluate the `SecondExpr`,  `firstExpr` should be `true` in the case of `&&` or `false` in the case of `||`.

### Exercices
**2.1-2** Rewrite the INSERTION-SORT procedure to sort into nonincreasing instead of non-decreasing order.

In [ ]:
function reversedInsertionSort(A::Array)
    for i in 2:size(A)[1]
        key = A[i]
        j = i - 1
        while j > 0 && A[j] < key
            A[j+1] = A[j]
            j -= 1
        end
        A[j+1] = key
    end
    return A
end
reversedInsertionSort([13, 7, 2, 30, 43])

**2.1-3** Consider the *searching problem*:   

**Input:** A sequence of $n$ numbers $A = \langle a_1, a_2,\ldots , a_n\rangle$ and a value $\nu$  
**Output:** An index $i$ such that $\nu = A[i]$ or `Nothing` if $\nu$ does not appear in $A$    
Write pseudocode for linear search, which scans through the sequence, looking
for $\nu$. Using a loop invariant, prove that your algorithm is correct. Make sure that your loop invariant fulfills the three necessary properties.

```julia
function SearchingProblem(A, nu)
    for i = 1 to A.length
        if A[i] == nu
            return i
        end
    end
    return None
end
```

**Initialization:** We start by checking that first value is not equal to $\nu$ before it nothing was proved yet  

**Maintenance:** Throughout the loops we check whether the value at current index is equal to $\nu$ in which case we return the index. Previous checked values remain as they are.

**Termination:** The algorithm terminates once a positive match is found or when all the elements in the array are exhausted.

**2.1-4** Consider the problem of adding two $n$-bit binary integers, stored in two n-element arrays $A$ and $B$. The sum of the two integers should be stored in binary form in an $(n+1)$-element array $C$ . State the problem formally and write pseudocode for adding the two integers. (the sum of two equal length bins outputs a n+1 length bin)

In [ ]:
function binInt(A::BitArray, B::BitArray)
    """Return the binary sum of two equal sized BitArrays.
    
    Process element wise the sum of the binary digits knowing that:
    A[i] + B[i] + carry = sum
      0  +   0  +   0   = 0
      0  +   1  +   0   = 1 + 0 + 0 = 1
      1  +   1  +   0   = 0 (carry = 1)
      1  +   1  +   1   = 1 (carry = 1)
    
    Notice also that the sum of two equal length bins always output a n+1
    length array because the first digit is 1. (Were 0 and the arrays wouldn't
    be equal) and 1 + 1 = 10
    """
    i = size(A)[2]  # == size(B)[2]; start at the end
    carry = 0
    C = []
    while i > 0
        s = A[i] + B[i] + carry
        if s == 2  # 1 + 1 + 0
            carry = 1
            s = 0  # add the "0" in "10"
        elseif s == 3  # 1 + 1 + 1
            carry = 1
            s = 1  # add the "1" in "11"
        else  # 0 + 0; 0 + 1; 1 + 0
            carry = 0
        end
        insert!(C, 1, s)
        i -= 1  # go backwards
    end
    insert!(C, 1, carry)  # Insert the last digit (should be a 1)
    return C
end

A = BitArray([1 1 1 0 0 0])
B = BitArray([1 0 1 0 1 1])

binInt(A, B)

## Analyzing algorithms

**RAM MODEL**  

An ideal & uniform framework to compare algorithms in an unbiased way.

Allowed operations:
- **Arithmetic:** add, substract, multiply, divide, remainder, floor & ceiling
- **Data movement:** load, store & copy
- **Control:** conditional & unconditional branch, subroutine call & return

Exponetiation is not a constant time operation in general, but in the case of shifting bits by $k$ positions, which means to murtiply by $2^k$ (notice that for negative $k$ this is dividing $k$ times by $2$)

Ram model doesn't consider memory hierarchy (they are hard to work with):  

<img src="https://upload.wikimedia.org/wikipedia/commons/0/0c/ComputerMemoryHierarchy.svg" alt="drawing" width="600px"/>

**Size of the input:**  
Size of the input can be tricky to define:
- It can be the number of elements: like in the insertion sort seen.
- It can be the number of bits: like multiplying two integers. 
- Also it can be defined by two numbers: like in graphs, edges & vertices

**Running times:**  
When calculating running times we assume the following:
- The running time of the algorithm is the sum of the running time of each statement
- Each statement takes a cost $c_i$ every time it is executed
- We usually pay attention to the worst case:
    - to ensure an upper bound.
    - For some algorithms the worst case occurs fairly often (like when searching a database one hits the *information not present*)
    - The average case is often roughly as bad as the worst case.
- Although sometimes the average case is useful (like with randomized algorithms)

**Order of growth:**  
Is the most significant exponent what really matters, though, that is, in a running time of $an^2+bn+c$, $bn+c$ remain insignificant as $n$ grows larger. So we write: 

$\Theta(n^2)$, pronounced theta of n-squared

### Exercises

Consider sorting n numbers stored in array $A$ by first finding the smallest element of $A$ and exchanging it with the element in $A[1]$. Then find the second smallest element of $A$, and exchange it with $A[2]$. Continue in this manner for the first $n-1$ elements of $A$. Write pseudocode for this algorithm, which is known as **selection sort.** What loop invariant does this algorithm maintain? Why does it need to run for only the first $n-1$ elements, rather than for all $n$ elements? Give the best-case and worst-case running times of selection sort in $\Theta$-notation.

**Details**  
- Iterate all the elements except the last one
- Assume that current element is actually the lowest number in the array and check all the elements forward.
- If a lower element is found set it as --temporally-- lowest
- When all the numbers after current elements are checked swap the lowest with the current and move on to the next.

**Questions:**
- *What loop invariant does this algorithm maintain?* -> all the elements before the current one are sorted because in each loop we search the lowest one and put it at the rightmost position. So on and so forth until the previous to the last one
- *Why does it need to run for only the first $n-1$ elements?* -> when finishing that should be the biggest number if the algorithm is correct
- *Best/worst case:* -> there's no best case nor worst case because always we have to iterate all the remaning elements in the array to ensure the current one is the smallest, so its running time is $\Theta(n^2)$ because of the nested loop 

In [ ]:
function selectionSort(A::Array)
    len = size(A)[1]
    for j in 1:len - 1
        i = j + 1
        lower = j  # you are currently the lower element if nothing happens
        while i <= len
            if A[i] < A[lower]
                lower = i
            end
            i += 1
        end
        if A[lower] != A[j]
            A[lower], A[j] = A[j], A[lower]
        end
    end
    return A
end

selectionSort([4, 5, 7, 3, 1, 2])

## Designing algorithms: Merge sort

For insertion sort we choose the **incremental approach**, now we'll use a different approach, **divide-and-conquer**

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e6/Merge_sort_algorithm_diagram.svg" alt="drawing" width="400px"/>

In [ ]:
function mergeSort(A::Array; verbose=true::Bool)
    # base case, an array with no elements or one element is sorted by default
    # This step stops the recursion, backpropagating the effect.
    if size(A)[1] <= 1
        verbose == true && print(
            "$A has one element: keep it until merge time. \n")
        return A
    end    
    
    # recursive case. Split the arrays two halves
    left, right = Int16[], Int16[]
    for (i, v) in enumerate(A)
        if i <= size(A)[1] / 2
            push!(left, v)
        else
            push!(right, v)
        end
    end
    verbose == true && println("Splitted $A into $left, $right")
    
    # And repeat above split until the array is left with one element where the
    # function will exit letting next step (mergeArrays()) to be executed.
    # Current iteration keeps waiting until last split is done.
    left = mergeSort(left, verbose=verbose)
    right = mergeSort(right, verbose=verbose)
    
    # Once all the elements are splitted start merging backwards
    return mergeArrays(left, right, verbose=verbose)
end

function mergeArrays(left::Array, right::Array; verbose=true::Bool)
    verbose == true && println("Merging arrays $left & $right:")

    # Loop invariant, initialization: in the beginning, the merged array is 
    # empty, so prior to the first iteration it contains 0 smallest elements
    # from left and right arrays
    mergedArray = Int16[]
    
    
    # Loop invariant, maintenance: over loops we choose the smallest of the
    # comparison between left & right and since they came sorted, mergedArray
    # holds over iterations
    while ~isempty(left) && ~isempty(right)
        r, l = right[1], left[1]
        if l <= r
            verbose == true && println("    $l is lower than $r => add $l")
            push!(mergedArray, left[1])
            left = left[2:end]  # pop!() removes last item
        else
            verbose == true && println("    $l is higher than $r => add $r")
            push!(mergedArray, right[1])
            right = right[2:end]
        end
    end
    
    # One and only one of those arrays won't be empty but its elements are
    # already ordered since the previous iterations, so just add them 
    ~isempty(left) && verbose == true && println(
        "    $left is not empty, exhaust it (already sorted)")
    ~isempty(right) && verbose == true && println(
        "    $right is not empty, exhaust it (already sorted)")
    while ~isempty(left)
        push!(mergedArray, left[1])
        left = left[2:end]
    end
    while ~isempty(right)
        push!(mergedArray, right[1])
        right = right[2:end]
    end
    
    # Loop invariant, termination: since we're left with empty sub-arrays, and 
    # we have kept sorting elements we must conclude that the merged array
    # contains both subarrays ordered
    
    verbose == true && println(" -> Merged array: $mergedArray")
    return mergedArray
end

mergeSort(Int16[38, 27, 43, 3, 9, 82, 10], verbose=true)
        

### Exercises

**2.3-4**
We can express insertion sort as a recursive procedure as follows. In order to sort $A[1..n]$, we recursively sort $A[1..n-1]$ and then insert $A[n]$ into the sorted array $A[1..n-1]$. Write a recurrence for the running time of this recursive version of insertion sort.

In [ ]:
function insertionSort(A::Array)
    for i in 1:length(A)
        key = A[i]
        j = i - 1
        while j > 0 && key < A[j]
            A[j+1] = A[j]
            j -= 1
        end
        A[j+1] = key
    end
    return A
end


function insertionSortRecursive(A::Array)
    if length(A) == 1
        return A
    end
    right = A[end]
    left = A[1:end - 1]
    left = insertionSortRecursive(left)
    push!(left, right)
    
    return insertionSort(left)
end

insertionSortRecursive([10, 2, 1, 7, 3])
    

**2.3-5**
Referring back to the searching problem (see Exercise 2.1-3), observe that if the sequence $A$ is sorted, we can check the midpoint of the sequence against $\nu$ and eliminate half of the sequence from further consideration. The binary search algorithm repeats this procedure, halving the size of the remaining portion of the sequence each time. Write pseudocode, either iterative or recursive, for binary search. Argue that the worst-case running time of binary search is ‚$\Theta(lg n)$.

In [ ]:
function binarySearchRecursive(A::Array, nu::Int64)
    if length(A) == 1 
        if A[1] == nu
            return true
        else
            return false
        end
    end
    
    mid_point = div(length(A), 2)
    if A[mid_point] > nu
        A = A[1:mid_point]
    elseif A[mid_point] < nu
        A = A[mid_point + 1:end]
    else
        # Midpoint is nu
        return true
    end
    return binarySearchRecursive(A, nu)
end

binarySearchRecursive(collect(1:100), 1)

In [ ]:
collect(1:10)

## Loop invariants

This is a short incursion into loop invariants because they seem to be quite useful.

[Source](https://yourbasic.org/algorithms/loop-invariants-explained/)

**Definition**  
> A loop invariant is a statement about program variables that is true before and after each iteration of a loop.

And the three properties seen before:
* **Initialization:** the condition holds before the first iteration
* **Maintenance:** the condition holds over loops. If the invariant is true before an iteration of the loop, it should be true also after the iteration.
* **Termination:** When the loop is terminated the invariant should tell us something useful, something that helps us understand the algorithm.

**The simplest example:**

In [ ]:
function sumFromZero(n::Int64)::Int64
    """Sum all the numbers from zero to n."""
    # Initialization: the sum at the beginning is zero and the loop starts at 1
    sum = 0
    i = 1
    
    # Maintenance: the loop invariant holds because we add i to the sum and 
    # increment it by one unit
    while i <= n
        sum += i
        i += 1
    end
    
    # Termination: since we have exhausted all the integers less or equal than
    # n we are left with sum = 0 +1 + 2 + 3 + ... + n
    
    return sum 
end
sumFromZero(10) == 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10

When designing loops, start by the termination clause, then adjust initialization and finally set maintenance

### 3-way partition algorithm (delayed)
*Delayed until reaching its chapter*

[source](https://yourbasic.org/golang/quicksort-optimizations/)

Is an optimization algortithm, that is, it's used to improve the efficiency of some other sorting algorithm like insertion sort

Overview:
* Pick an element p, called a pivot, from the list.
* Partition the list so that
    * all elements less than p come first,
    * all elements greater than p come last, 
    * elements equal to p go into the middle.
* Recursively apply the above steps to the sublists of small and large elements.
* For short sublists, use a simpler sorting algorithm.


In [ ]:
using StatsBase
using Statistics
A = sample(1:10000, 50, replace = false)

function insertionSort(A::Array)::Array
    for i in 2:size(A)[1]
        key = A[i]
        j = i - 1
        while j > 0 && key < A[j]
            A[j + 1] = A[j]
            j -= 1
        end
        A[j+1] = key
    end
    return A
end

function pivot(v::Array)::Int64
    
end

function partition(v::Array, p::Int64)            
end

In [ ]:
function quickSort(A::Array)::Array
    if size(A)[1] < 20
        println("order $A by insertion sort")
        return insertionSort(A)
    end
    
    p = pivot(A)
    low, high = partition(v, p)
    
    quickSort
        
    # Initialization: in the beginning all the arrays are empty
    left, middle, right = [], [], []
    
    
    p = div(size(A)[1], 2)
    for n in A
        pivot_value = A[p]
        if n < A[p]
            println("$n is less than $pivot_value")
            push!(left, n)
        elseif n > A[p]
            push!(right, n)
        else
            push!(middle, n)
        end
    end
    left = partition(left)
#     middle = partition(middle)
    rigth = partition(right)
    
    return vcat(left, middle, right)
end


# check everything is ok
A = partition(A)
for i in 1:size(A)[1] - 1
    cur, nxt = A[i], A[i+1]
    if cur > nxt
        println("$cur > $nxt at $i")
    end
end

# A

# Growth of functions

# Divide-and-conquer

# Probabilistic Analysis and Randomized Algorithms